In [71]:
from flask import *
from datetime import datetime
from flask_bcrypt import Bcrypt 
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user  
import pandas as pd
import os

In [72]:
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///candidates.sqlite"
app.config["SECRET_KEY"] = "2210993778"
db = SQLAlchemy()

In [73]:
# Define the Candidate model
class Candidate(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(100), nullable=False)
    score = db.Column(db.Integer, nullable=False)

In [74]:
# Add an event listener to the database session
@db.event.listens_for(Candidate, 'after_insert')
@db.event.listens_for(Candidate, 'after_update')
@db.event.listens_for(Candidate, 'after_delete')
def after_candidate_changes(mapper, connection, candidate):
    candidates = Candidate.query.all()
    df = pd.DataFrame([(c.id, c.name, c.email, (c.score/20)*100) for c in candidates], columns=['id', 'name', 'email', 'score'])
    df.to_csv('candidates.csv', index=False)

In [75]:
db.init_app(app)

with app.app_context():
    db.create_all()

In [76]:
# Read questions from CSV file and shuffle them
questions_df = pd.read_csv("data\\Questions.csv")
questions = questions_df.to_dict('records')  # Convert DataFrame to a list of dictionaries

In [77]:
# Display the quiz interface
@app.route('/')
def index():
    return render_template('index1.html')

@app.route('/quiz')
def quiz():
    return render_template('quiz.html', questions=questions)

# Handle form submission
@app.route('/submit', methods=['POST'])
def submit():
    name = request.form['name']
    email = request.form['email']
    score = calculate_score(request.form)

    # Check if a candidate with the same email already exists
    candidate = Candidate.query.filter_by(email=email).first()

    if candidate:
        # Update the existing candidate's score
        candidate.score = score
    else:
        # Create a new candidate
        candidate = Candidate(name=name, email=email, score=score)
        db.session.add(candidate)

    # Store candidate details and score in the database
    db.session.commit()

    return render_template('submit.html')

# Calculate candidate score based on answers
def calculate_score(form_data):
    score = 0
    for question in questions:
        answer_key = f"answer_{question['Question ID']}"
        if form_data.get(answer_key) == str(question['Correct Answer']):
            score += 1
    return score

In [78]:
# Display all candidates and their marks in percentage
@app.route('/leaderboard')
def leaderboard():
    candidates = Candidate.query.order_by(Candidate.score.desc()).all()
    ranked_candidates = generate_ranks(candidates)
    return render_template('leaderboard.html', candidates=ranked_candidates)

# Function to generate ranks for candidates
def generate_ranks(candidates):
    ranked_candidates = []
    rank = 1
    prev_score = None

    # Sort candidates by score in descending order, and by name in ascending order
    sorted_candidates = sorted(candidates, key=lambda c: (-c.score, c.name))

    for candidate in sorted_candidates:
        if candidate.score != prev_score:
            rank = len(ranked_candidates) + 1
            prev_score = candidate.score
        ranked_candidates.append({'rank': rank, 'candidate': candidate})

    return ranked_candidates

In [79]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/May/2024 18:06:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2024 18:06:12] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2024 18:06:12] "GET /static/logobg.png HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2024 18:06:16] "GET /quiz HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2024 18:06:16] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2024 18:06:16] "GET /static/logobg.png HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2024 18:06:18] "GET /leaderboard HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2024 18:06:18] "GET /static/leaderboard.css HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2024 18:06:18] "GET /static/logobg.png HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2024 18:06:21] "GET /leaderboard HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2024 18:06:21] "GET /static/leaderboard.css HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2024 18:06:21] "GET /static/logobg.png HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2024 18:06:22] "GET /leaderboa